# M2 — Primer pipeline Olive end-to-end (ONNX → optimizaciones → validación)

Este notebook continúa el curso **Curso Olive + Python (VS Code)**.

## Objetivo
1. Tomar un modelo **ONNX** (el `models/add_const.onnx` del M1).
2. Ejecutar un **workflow de Olive** con un archivo de configuración (`.json`) usando `python -m olive run`.
3. Revisar artefactos en `outputs/` y **validar** que el modelo optimizado mantiene el resultado.

## Prerrequisitos
- VS Code con notebooks Jupyter.
- Entorno `.venv` activado.
- Paquetes instalados en el venv: `olive-ai`, `onnxruntime`, `onnx`, `numpy`.

Estructura recomendada del repo:
```
notebooks/
models/
outputs/
```


In [ ]:
# Celda 1 — Comprobación de entorno (versiones)
import sys, subprocess
import onnxruntime as ort

print("Python:", sys.version)
print("ONNX Runtime:", ort.get_version_string())
print("Available providers:", ort.get_available_providers())

# (Obligatorio en el curso) validar versión de Olive y Python
subprocess.run([sys.executable, "-m", "pip", "show", "olive-ai"], check=False)


## Explicación breve

Olive ejecuta una **secuencia de passes** (transformaciones/optimizaciones) definida en un archivo JSON/YAML.

En este M2 vamos a:
- Configurar un **sistema local** (CPU).
- Aplicar dos passes sencillos sobre un modelo ONNX:
  - `OnnxOpVersionConversion` (para fijar opset objetivo).
  - `OnnxModelOptimizer` (optimizaciones/fusiones estándar sobre el grafo).
- Guardar el resultado en `outputs/m2_add_const/` y comprobar inferencia con ONNX Runtime.


# Práctica en Notebook

## 1) Asegurar el modelo de entrada (M1)

Si ya tienes `models/add_const.onnx`, esta celda no modifica nada.
Si no existe, lo crea (modelo: `Y = X + 1`).


In [ ]:
from pathlib import Path
import numpy as np
import onnx
from onnx import TensorProto, helper, numpy_helper

Path("../models").mkdir(exist_ok=True)
Path("outputs").mkdir(exist_ok=True)

model_path = Path("../models") / "add_const.onnx"

if not model_path.exists():
    # 1) Definir IO del modelo
    X = helper.make_tensor_value_info("X", TensorProto.FLOAT, [None])  # vector 1D de tamaño variable
    Y = helper.make_tensor_value_info("Y", TensorProto.FLOAT, [None])

    # 2) Constante C (float32)
    C_value = np.array([1.0], dtype=np.float32)
    C = numpy_helper.from_array(C_value, name="C")

    # 3) Nodo: Add(X, C) -> Y
    node = helper.make_node("Add", inputs=["X", "C"], outputs=["Y"])

    # 4) Grafo + Modelo
    graph = helper.make_graph(nodes=[node], name="add-const", inputs=[X], outputs=[Y], initializer=[C])

    # Crear el modelo con opset 11 e IR 11 (compatibilidad amplia con runtimes)
    opset = [helper.make_operatorsetid("", 11)]
    model = helper.make_model(graph, producer_name="m2-lab", opset_imports=opset)
    model.ir_version = 11

    # 5) Validar y guardar
    onnx.checker.check_model(model)
    onnx.save_model(model, str(model_path))

print("Input model:", model_path.resolve())


## 2) Crear el config JSON de Olive

Vamos a escribir un archivo `outputs/m2_run_config.json` y luego ejecutar:

```
python -m olive run --config outputs/m2_run_config.json
```

Nota: usamos `python -m olive` para no depender de que `olive` esté en el PATH.


In [ ]:
import json
from pathlib import Path

# Detectar raíz del proyecto (si estás en notebooks/, sube 1)
root = Path.cwd()
if not (root / "models" / "add_const.onnx").exists() and (root.parent / "models" / "add_const.onnx").exists():
    root = root.parent

model_path = root / "models" / "add_const.onnx"
run_config_path = root / "outputs" / "m2_run_config.json"
output_dir = root / "outputs" / "m2_add_const"
cache_dir = root / "outputs" / "m2_cache"

(root / "outputs").mkdir(exist_ok=True)

config = {
    "workflow_id": "m2_add_const_e2e",
    "input_model": {
        "type": "ONNXModel",  # Nota: en versiones recientes de Olive es ONNXModel, no ONNXModelHandler
        "config": {
            "model_path": str(model_path),
        },
    },
    "systems": {
        "local_system": {
            "type": "LocalSystem",
            "config": {
                "accelerators": [
                    {
                        "device": "cpu",
                        "execution_providers": ["CPUExecutionProvider"],
                    }
                ]
            },
        }
    },
    "passes": {
        "opset_to_11": {
            "type": "OnnxOpVersionConversion",
            "config": {"target_opset": 11},
        },
        "onnx_optimize": {
            "type": "OnnxPeepholeOptimizer",  # Nota: OnnxModelOptimizer ya no existe, usar OnnxPeepholeOptimizer
            "config": {},
        },
    },
    "engine": {
        "host": "local_system",
        "target": "local_system",
        "cache_dir": str(cache_dir),
        "output_dir": str(output_dir),
        "log_severity_level": 0,
        "evaluate_input_model": False,
    },
}

run_config_path.write_text(json.dumps(config, indent=2), encoding="utf-8")
print("Wrote:", run_config_path)
print("Model:", model_path)
print("Output:", output_dir)

## 5) Validación: inferencia con ONNX Runtime (baseline vs optimizado)

- Cargamos el modelo original (`models/add_const.onnx`)
- Cargamos un modelo ONNX generado por Olive
- Ejecutamos con el mismo input y comprobamos que el resultado coincide.


In [ ]:
import numpy as np
import onnxruntime as ort
from pathlib import Path

baseline = Path("../models") / "add_const.onnx"

# Buscar archivos .onnx generados por Olive
onnx_files = list(Path("../outputs/m2_add_const").rglob("*.onnx"))

if not onnx_files:
    raise FileNotFoundError("No se encontraron archivos .onnx en outputs/m2_add_const. Revisa el log de Olive.")

# Elegimos el primer .onnx encontrado. Si Olive genera varios, puedes cambiar este criterio.
optimized = onnx_files[0]

def run_ort(model_path: Path, x: np.ndarray) -> np.ndarray:
    sess = ort.InferenceSession(str(model_path), providers=["CPUExecutionProvider"])
    input_name = sess.get_inputs()[0].name
    output_name = sess.get_outputs()[0].name
    y = sess.run([output_name], {input_name: x})[0]
    return y

x = np.array([10, 20, 30], dtype=np.float32)
y_base = run_ort(baseline, x)
y_opt = run_ort(optimized, x)

print("baseline:", baseline.resolve())
print("optimized:", optimized.resolve())
print("x:", x)
print("y_base:", y_base)
print("y_opt :", y_opt)
print("All close:", np.allclose(y_base, y_opt))

## Verificación (qué debe salir)

- `returncode: 0` en la celda de ejecución de Olive.
- En `outputs/m2_add_const/` debe aparecer al menos un `.onnx`.
- En la validación final: `All close: True`.


## Errores comunes y diagnóstico

1) **No aparece ningún `.onnx` en outputs/**
   - Revisar el `stderr`/`stdout` de la celda 3.
   - Confirmar que el `model_path` existe y apunta al ONNX correcto.

2) **El pass no existe / nombre incorrecto**
   - Pasa cuando tu versión instalada difiere. En ese caso, mira la ayuda / docs de tu versión, o pega el error.

3) **Problemas de proveedores ORT**
   - Aquí forzamos `CPUExecutionProvider` para hacerlo reproducible.

## Siguiente paso (M3)
En M3 vamos a introducir **cuantización** (por ejemplo, `OnnxDynamicQuantization`) y hablaremos de **calidad vs rendimiento**.
